**Note: All experiments have been run on PL data only**

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
import json
import operator
from matplotlib.ticker import FuncFormatter
from matplotlib.patches import Ellipse
import base64
from collections import defaultdict
import sys,os
import math
import random
import matplotlib.pylab as pyl
import itertools
import pickle
import swifter
import warnings
from unidecode import unidecode
from itertools import chain
from multiprocessing import  Pool

warnings.filterwarnings('ignore')

C:\Users\Anuj\Anaconda3\lib\site-packages\dask\dataframe\utils.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
pd.set_option('max_colwidth', 999)

In [3]:
pd.set_option('display.max_columns', 1000)
pd.set_option("display.max_rows", 3000)
warnings.filterwarnings('ignore')

# Competitions 

- area: it denotes the geographic area associated with the league as a sub-document, using the ISO 3166-1 specification (https://www.iso.org/iso-3166-country-codes.html);
- format: the format of the competition. All competitions for clubs have value "Domestic league". The competitions for national teams have value "International cup";
- name: the official name of the competition (e.g., Italian first division, Spanish first division, World Cup, etc.);
- type: the typology of the competition. It is "club" for the competitions for clubs and "international" for the competitions for national teams (World Cup 2018, European Cup 2016);
- wyId: the unique identifier of the competition, assigned by Wyscout.

In [3]:
comps = pd.read_json('../data/competitions.json')
comps = pd.DataFrame(comps)
comps

,name,wyId,format,area,type
0,Italian first division,524,Domestic league,"{'name': 'Italy', 'id': '380', 'alpha3code': '...",club
1,English first division,364,Domestic league,"{'name': 'England', 'id': '0', 'alpha3code': '...",club
2,Spanish first division,795,Domestic league,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
3,French first division,412,Domestic league,"{'name': 'France', 'id': '250', 'alpha3code': ...",club
4,German first division,426,Domestic league,"{'name': 'Germany', 'id': '276', 'alpha3code':...",club
5,European Championship,102,International cup,"{'name': '', 'id': 0, 'alpha3code': 'XEU', 'al...",international
6,World Cup,28,International cup,"{'name': '', 'id': 0, 'alpha3code': 'XWO', 'al...",international


# Matches 

- competitionId: the identifier of the competition to which the match belongs to. It is a integer and refers to the field "wyId" of the competition document;
- date and dateutc: the former specifies date and time when the match starts in explicit format (e.g., May 20, 2018 at 8:45:00 PM GMT+2), the latter contains the same information but in the compact format YYYY-MM-DD hh:mm:ss;
- duration: the duration of the match. It can be "Regular" (matches of regular duration of 90 minutes + stoppage time), "ExtraTime" (matches with supplementary times, as it may happen for matches in continental or international competitions), or "Penalities" (matches which end at penalty kicks, as it may happen for continental or international competitions);
- gameweek: the week of the league, starting from the beginning of the league;
- label: contains the name of the two clubs and the result of the match (e.g., "Lazio - Internazionale, 2 - 3");
- roundID: indicates the match-day of the competition to which the match belongs to. During a competition for soccer clubs, each of the participating clubs plays against each of the other clubs twice, once at home and once away. The matches are organized in match-days: all the matches in match-day i are played before the matches in match-day i + 1, even tough some matches can be anticipated or postponed to facilitate players and clubs participating in Continental or Intercontinental competitions. During a competition for national teams, the "roundID" indicates the stage of the competition (eliminatory round, round of 16, quarter finals, semifinals, final);
- seasonId: indicates the season of the match;
- status: it can be "Played" (the match has officially finished), "Cancelled" (the match has been canceled for some reason), "Postponed" (the match has been postponed and no new date and time is available yet) or "Suspended" (the match has been suspended and no new date and time is available yet);
- venue: the stadium where the match was held (e.g., "Stadio Olimpico");
- winner: the identifier of the team which won the game, or 0 if the match ended with a draw;
- wyId: the identifier of the match, assigned by Wyscout;
- teamsData: it contains several subfields describing information about each team that is playing that match: such as lineup, bench composition, list of substitutions, coach and scores:
- hasFormation: it has value 0 if no formation (lineups and benches) is present, and 1 otherwise;
- score: the number of goals scored by the team during the match (not counting penalties);
- scoreET: the number of goals scored by the team during the match, including the extra time (not counting penalties);
- scoreHT: the number of goals scored by the team during the first half of the match;
- scoreP: the total number of goals scored by the team after the penalties;
- side: the team side in the match (it can be "home" or "away");
- teamId: the identifier of the team;
- coachId: the identifier of the team's coach;
- bench: the list of the team's players that started the match in the bench and some basic statistics about their performance during the match (goals, own goals, cards);
- lineup: the list of the team's players in the starting lineup and some basic statistics about their performance during the match (goals, own goals, cards);
- substitutions: the list of team's substitutions during the match, describing the players involved and the minute of the substitution.

In [4]:
matches = pd.read_json('../data/matches/matches_England.json')
matches = pd.DataFrame(matches)
print(matches.shape)
display(matches)

(380, 14)


,status,roundId,gameweek,teamsData,seasonId,dateutc,winner,venue,wyId,label,date,referees,duration,competitionId
0,Played,4405654,38,"{'1646': {'scoreET': 0, 'coachId': 8880, 'side...",181150,2018-05-13 14:00:00,1659,Turf Moor,2500089,"Burnley - AFC Bournemouth, 1 - 2","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 385705, 'role': 'referee'}, {'r...",Regular,364
1,Played,4405654,38,"{'1628': {'scoreET': 0, 'coachId': 8357, 'side...",181150,2018-05-13 14:00:00,1628,Selhurst Park,2500090,"Crystal Palace - West Bromwich Albion, 2 - 0","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 381851, 'role': 'referee'}, {'r...",Regular,364
2,Played,4405654,38,"{'1609': {'scoreET': 0, 'coachId': 7845, 'side...",181150,2018-05-13 14:00:00,1609,The John Smith's Stadium,2500091,"Huddersfield Town - Arsenal, 0 - 1","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 384965, 'role': 'referee'}, {'r...",Regular,364
3,Played,4405654,38,"{'1651': {'scoreET': 0, 'coachId': 8093, 'side...",181150,2018-05-13 14:00:00,1612,Anfield,2500092,"Liverpool - Brighton & Hove Albion, 4 - 0","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 385704, 'role': 'referee'}, {'r...",Regular,364
4,Played,4405654,38,"{'1644': {'scoreET': 0, 'coachId': 93112, 'sid...",181150,2018-05-13 14:00:00,1611,Old Trafford,2500093,"Manchester United - Watford, 1 - 0","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 381853, 'role': 'referee'}, {'r...",Regular,364
5,Played,4405654,38,"{'1613': {'scoreET': 0, 'coachId': 210700, 'si...",181150,2018-05-13 14:00:00,1613,St. James' Park,2500094,"Newcastle United - Chelsea, 3 - 0","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 384888, 'role': 'referee'}, {'r...",Regular,364
6,Played,4405654,38,"{'1625': {'scoreET': 0, 'coachId': 267136, 'si...",181150,2018-05-13 14:00:00,1625,St. Mary's Stadium,2500095,"Southampton - Manchester City, 0 - 1","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 385911, 'role': 'referee'}, {'r...",Regular,364
7,Played,4405654,38,"{'10531': {'scoreET': 0, 'coachId': 32573, 'si...",181150,2018-05-13 14:00:00,1639,Liberty Stadium,2500096,"Swansea City - Stoke City, 1 - 2","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 378952, 'role': 'referee'}, {'r...",Regular,364
8,Played,4405654,38,"{'1631': {'scoreET': 0, 'coachId': 209010, 'si...",181150,2018-05-13 14:00:00,1624,Wembley Stadium,2500097,"Tottenham Hotspur - Leicester City, 5 - 4","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 378951, 'role': 'referee'}, {'r...",Regular,364
9,Played,4405654,38,"{'1623': {'scoreET': 0, 'coachId': 8541, 'side...",181150,2018-05-13 14:00:00,1633,London Stadium,2500098,"West Ham United - Everton, 3 - 1","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 408156, 'role': 'referee'}, {'r...",Regular,364


Sample 'TeamsData' row entry:

# Events

- eventId: the identifier of the event's type. Each eventId is associated with an event name (see next point);
- eventName: tteamIdhe name of the event's type. There are seven types of events: pass, foul, shot, duel, free kick, offside and touch;
- subEventId: the identifier of the subevent's type. Each subEventId is associated with a subevent name (see next point);
- subEventName: the name of the subevent's type. Each event type is associated with a different set of subevent types;
- tags: a list of event tags, each one describes additional information about the event (e.g., accurate). Each event type is associated with a different set of tags;
- eventSec: the time when the event occurs (in seconds since the beginning of the current half of the match);
- id: a unique identifier of the event;
- matchId: the identifier of the match the event refers to. The identifier refers to the field "wyId" in the match dataset;
- matchPeriod: the period of the match. It can be "1H" (first half of the match), "2H" (second half of the match), "E1" (first extra time), "E2" (second extra time) or "P" (penalties time);
- playerId: the identifier of the player who generated the event. The identifier refers to the field "wyId" in a player dataset;
- positions: the origin and destination positions associated with the event. Each position is a pair of coordinates (x, y). The x and y coordinates are always in the range [0, 100] and indicate the percentage of the field from the perspective of the attacking team. In particular, the value of the x coordinate indicates the event's nearness (in percentage) to the opponent's goal, while the value of the y coordinates indicates the event's nearness (in percentage) to the right side of the field;
- teamId: the identifier of the player's team. The identifier refers to the field "wyId" in the team dataset.

In [5]:
events = pd.read_json('../data/events/events_England.json')
events = pd.DataFrame(events)
print(events.shape)
display(events)

(643150, 12)


,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id
0,8,Simple pass,[{'id': 1801}],25413,"[{'y': 49, 'x': 49}, {'y': 78, 'x': 31}]",2499719,Pass,1609,1H,2.758649,85,177959171
1,8,High pass,[{'id': 1801}],370224,"[{'y': 78, 'x': 31}, {'y': 75, 'x': 51}]",2499719,Pass,1609,1H,4.946850,83,177959172
2,8,Head pass,[{'id': 1801}],3319,"[{'y': 75, 'x': 51}, {'y': 71, 'x': 35}]",2499719,Pass,1609,1H,6.542188,82,177959173
3,8,Head pass,[{'id': 1801}],120339,"[{'y': 71, 'x': 35}, {'y': 95, 'x': 41}]",2499719,Pass,1609,1H,8.143395,82,177959174
4,8,Simple pass,[{'id': 1801}],167145,"[{'y': 95, 'x': 41}, {'y': 88, 'x': 72}]",2499719,Pass,1609,1H,10.302366,85,177959175
...,...,...,...,...,...,...,...,...,...,...,...,...
643145,5,Ball out of the field,[],0,"[{'y': 32, 'x': 0}, {'y': 100, 'x': 100}]",2500098,Interruption,1623,2H,2796.732525,50,251596409
643146,3,Corner,"[{'id': 302}, {'id': 801}, {'id': 1801}]",70965,"[{'y': 100, 'x': 100}, {'y': 47, 'x': 88}]",2500098,Free Kick,1633,2H,2829.821084,30,251596232
643147,1,Air duel,"[{'id': 701}, {'id': 1802}]",7919,"[{'y': 53, 'x': 12}, {'y': 50, 'x': 14}]",2500098,Duel,1623,2H,2831.211419,10,251596410
643148,1,Air duel,"[{'id': 703}, {'id': 1801}]",8005,"[{'y': 47, 'x': 88}, {'y': 50, 'x': 86}]",2500098,Duel,1633,2H,2832.434399,10,251596234


In [6]:
print(len(events.matchId.unique()))
print(matches.shape)

380
(380, 14)


In [7]:
events.matchId.value_counts().mean()

1692.5

Approximately, every 3.19 seconds, an event activity is registered

# Player 

- birthArea: geographic information about the player's birth area;
- birthDate: the birth date of the player, in the format "YYYY-MM-DD";
- currentNationalTeamId: the identifier of the national team where the players currently plays;
- currentTeamId: the identifier of the team where the player plays for. The identifier refers to the field "wyId" in a team document;
- firstName: the first name of the player;
- lastName: the last name of the player;
- foot: the preferred foot of the player;
- height: the height of the player (in centimeters);
- middleName: the middle name (if any) of the player;
- passportArea: the geographic area associated with the player's current passport;
- role: the main role of the player. It is a subdocument containing the role's name and two abbreviations of it;
- shortName2: the short name of the player;
- weight: the weight of the player (in kilograms);
- wyId: the identifier of the player, assigned by Wyscout.

In [43]:
players = pd.read_json('../data/players.json')
players = pd.DataFrame(players)
print(players.shape)
display(players)

(3603, 14)


,passportArea,weight,firstName,middleName,lastName,currentTeamId,birthDate,height,role,birthArea,wyId,foot,shortName,currentNationalTeamId
0,"{'name': 'Turkey', 'id': '792', 'alpha3code': ...",78,Harun,,Tekin,4502,1989-06-17,187,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'Turkey', 'id': '792', 'alpha3code': ...",32777,right,H. Tekin,4687
1,"{'name': 'Senegal', 'id': '686', 'alpha3code':...",73,Malang,,Sarr,3775,1999-01-23,182,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'France', 'id': '250', 'alpha3code': ...",393228,left,M. Sarr,4423
2,"{'name': 'France', 'id': '250', 'alpha3code': ...",72,Over,,Mandanda,3772,1998-10-26,176,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'France', 'id': '250', 'alpha3code': ...",393230,,O. Mandanda,null
3,"{'name': 'Senegal', 'id': '686', 'alpha3code':...",82,Alfred John Momar,,N'Diaye,683,1990-03-06,187,"{'code2': 'MD', 'code3': 'MID', 'name': 'Midfi...","{'name': 'France', 'id': '250', 'alpha3code': ...",32793,right,A. N'Diaye,19314
4,"{'name': 'France', 'id': '250', 'alpha3code': ...",84,Ibrahima,,Konat\u00e9,2975,1999-05-25,192,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'France', 'id': '250', 'alpha3code': ...",393247,right,I. Konat\u00e9,null
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3598,"{'name': 'Tunisia', 'id': 788, 'alpha3code': '...",72,Ali,,Ma\u00e2loul,16041,1990-01-01,175,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'Tunisia', 'id': 788, 'alpha3code': '...",120839,left,A. Ma\u00e2loul,null
3599,"{'name': 'Peru', 'id': 604, 'alpha3code': 'PER...",76,Carlos Alberto,,C\u00e1ceda Oyaguez,15591,1991-09-27,183,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'Peru', 'id': 604, 'alpha3code': 'PER...",114736,right,C. C\u00e1ceda,null
3600,"{'name': 'Peru', 'id': 604, 'alpha3code': 'PER...",78,Miguel Gianpierre,,Araujo Blanco,12072,1994-10-24,179,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'Peru', 'id': 604, 'alpha3code': 'PER...",114908,right,M. Araujo,null
3601,"{'name': 'Morocco', 'id': 504, 'alpha3code': '...",70,Ahmed Reda,,Tagnaouti,16183,1996-04-05,182,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'Morocco', 'id': 504, 'alpha3code': '...",285583,right,A. Tagnaouti,null


# Events + Player DF 

In [7]:
events_com = pd.merge(right=players, left=events, right_on='wyId', left_on='playerId', how='left')
print(events_com.shape)
display(events_com)

(643150, 26)


,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,passportArea,weight,firstName,middleName,lastName,currentTeamId,birthDate,height,role,birthArea,wyId,foot,shortName,currentNationalTeamId
0,8,Simple pass,[{'id': 1801}],25413,"[{'y': 49, 'x': 49}, {'y': 78, 'x': 31}]",2499719,Pass,1609,1H,2.758649,85,177959171,"{'name': 'Guadeloupe', 'id': 312, 'alpha3code'...",73.0,Alexandre,,Lacazette,1609,1991-05-28,175.0,"{'code2': 'FW', 'code3': 'FWD', 'name': 'Forwa...","{'name': 'France', 'id': 250, 'alpha3code': 'F...",25413.0,right,A. Lacazette,null
1,8,High pass,[{'id': 1801}],370224,"[{'y': 78, 'x': 31}, {'y': 75, 'x': 51}]",2499719,Pass,1609,1H,4.946850,83,177959172,"{'name': 'England', 'id': 0, 'alpha3code': 'XE...",75.0,Rob,,Holding,1609,1995-09-20,189.0,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'England', 'id': 0, 'alpha3code': 'XE...",370224.0,right,R. Holding,null
2,8,Head pass,[{'id': 1801}],3319,"[{'y': 75, 'x': 51}, {'y': 71, 'x': 35}]",2499719,Pass,1609,1H,6.542188,82,177959173,"{'name': 'Germany', 'id': '276', 'alpha3code':...",76.0,Mesut,,\u00d6zil,1609,1988-10-15,180.0,"{'code2': 'MD', 'code3': 'MID', 'name': 'Midfi...","{'name': 'Germany', 'id': '276', 'alpha3code':...",3319.0,left,M. \u00d6zil,3148
3,8,Head pass,[{'id': 1801}],120339,"[{'y': 71, 'x': 35}, {'y': 95, 'x': 41}]",2499719,Pass,1609,1H,8.143395,82,177959174,"{'name': 'Egypt', 'id': '818', 'alpha3code': '...",70.0,Mohamed Naser,,Elsayed Elneny,1609,1992-07-11,180.0,"{'code2': 'MD', 'code3': 'MID', 'name': 'Midfi...","{'name': 'Egypt', 'id': '818', 'alpha3code': '...",120339.0,right,Mohamed Elneny,16129
4,8,Simple pass,[{'id': 1801}],167145,"[{'y': 95, 'x': 41}, {'y': 88, 'x': 72}]",2499719,Pass,1609,1H,10.302366,85,177959175,"{'name': 'Spain', 'id': 724, 'alpha3code': 'ES...",74.0,H\u00e9ctor,,Beller\u00edn Moruno,1609,1995-03-19,178.0,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'Spain', 'id': 724, 'alpha3code': 'ES...",167145.0,right,Beller\u00edn,null
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643145,5,Ball out of the field,[],0,"[{'y': 32, 'x': 0}, {'y': 100, 'x': 100}]",2500098,Interruption,1623,2H,2796.732525,50,251596409,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
643146,3,Corner,"[{'id': 302}, {'id': 801}, {'id': 1801}]",70965,"[{'y': 100, 'x': 100}, {'y': 47, 'x': 88}]",2500098,Free Kick,1633,2H,2829.821084,30,251596232,"{'name': 'Angola', 'id': '24', 'alpha3code': '...",67.0,Jo\u00e3o M\u00e1rio,,Naval da Costa Eduardo,3161,1993-01-19,173.0,"{'code2': 'MD', 'code3': 'MID', 'name': 'Midfi...","{'name': 'Portugal', 'id': '620', 'alpha3code'...",70965.0,right,Jo\u00e3o M\u00e1rio,9905
643147,1,Air duel,"[{'id': 701}, {'id': 1802}]",7919,"[{'y': 53, 'x': 12}, {'y': 50, 'x': 14}]",2500098,Duel,1623,2H,2831.211419,10,251596410,"{'name': 'England', 'id': 0, 'alpha3code': 'XE...",82.0,Michael,,Keane,1623,1993-01-11,188.0,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'England', 'id': 0, 'alpha3code': 'XE...",7919.0,right,M. Keane,2413
643148,1,Air duel,"[{'id': 703}, {'id': 1801}]",8005,"[{'y': 47, 'x': 88}, {'y': 50, 'x': 86}]",2500098,Duel,1633,2H,2832.434399,10,251596234,"{'name': 'Wales', 'id': 0, 'alpha3code': 'XWA'...",90.0,James Michael,,Collins,null,1983-08-23,193.0,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'Wales', 'id': 0, 'alpha3code': 'XWA'...",8005.0,right,J. Collins,null


In [8]:
#Those playerIds from df events that dont have an entry in df players
noplayer = events_com[events_com['firstName'].isna().values]
noplayer

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,passportArea,weight,firstName,middleName,lastName,currentTeamId,birthDate,height,role,birthArea,wyId,foot,shortName,currentNationalTeamId
8,1,Air duel,"[{'id': 703}, {'id': 1801}]",0,"[{'y': 85, 'x': 61}, {'y': 80, 'x': 67}]",2499719,Duel,1609,1H,14.765321,10,177961218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,1,Ground loose ball duel,"[{'id': 701}, {'id': 1802}]",0,"[{'y': 78, 'x': 33}, {'y': 85, 'x': 29}]",2499719,Duel,1609,1H,38.191874,13,177961219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,3,Free Kick,[{'id': 1801}],0,"[{'y': 44, 'x': 65}, {'y': 31, 'x': 65}]",2499719,Free Kick,1609,1H,74.737141,31,177959198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,1,Ground defending duel,"[{'id': 502}, {'id': 701}, {'id': 1802}]",0,"[{'y': 86, 'x': 4}, {'y': 97, 'x': 10}]",2499719,Duel,1631,1H,85.074043,12,177961220,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,1,Ground attacking duel,"[{'id': 701}, {'id': 1802}]",0,"[{'y': 27, 'x': 89}, {'y': 27, 'x': 95}]",2499719,Duel,1631,1H,152.127886,11,177961221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643080,5,Ball out of the field,[],0,"[{'y': 100, 'x': 51}, {'y': 100, 'x': 100}]",2500098,Interruption,1623,2H,2589.146520,50,251596374,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
643101,3,Free Kick,[{'id': 1801}],0,"[{'y': 87, 'x': 40}, {'y': 80, 'x': 56}]",2500098,Free Kick,1623,2H,2665.580593,31,251596391,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
643117,1,Ground defending duel,"[{'id': 502}, {'id': 701}, {'id': 1802}]",0,"[{'y': 34, 'x': 7}, {'y': 28, 'x': 5}]",2500098,Duel,1623,2H,2712.941293,12,251596987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
643140,3,Free Kick,[{'id': 1801}],0,"[{'y': 81, 'x': 70}, {'y': 70, 'x': 59}]",2500098,Free Kick,1633,2H,2780.300522,31,251596224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
noplayer.subEventName.value_counts()

Ball out of the field     27326
Ground defending duel      6206
Ground attacking duel      4841
Ground loose ball duel     4096
Air duel                   2972
Free Kick                  1550
Throw in                    652
Whistle                     201
Foul                         59
Goal kick                    46
Touch                        40
Corner                       28
Launch                        3
Simple pass                   3
                              3
Shot                          1
Hand foul                     1
Reflexes                      1
Free kick cross               1
Clearance                     1
Name: subEventName, dtype: int64

These entries are the interruptions that dont need a player (mostly, rest are data collection anomalies)

***Q: 'tags' values meaning in df events?***
 [describes additional information about the event (e.g., accurate)]

In [ ]:
events_com = events_com.dropna()
events_com = events_com.reset_index(drop=True)

# Extracting Player Position from Match Lineup 

## Extracting Defender Position 

In [14]:
defence = pd.read_csv('../data/PL1718Defense.csv')

In [ ]:
defence.drop(columns = ['Unnamed: 0'], inplace=True)

In [16]:
def clean_tokenize(s):
    s = s.replace(",", ' ')
    s = s.replace("[", '')
    s = s.replace("]", '')
    s = s.replace("'", '')
    s = s.split()
    return s

In [ ]:
defence['home_team_defense'] = defence['home_team_defense'].apply(lambda x: clean_tokenize(x))
defence['away_team_defense'] = defence['away_team_defense'].apply(lambda x: clean_tokenize(x))

In [27]:
def add_home_posn(df):
    for i in range(0, len(df)):
        if(len(df['home_team_defense'][i])==4):
            df['home_RB'][i] = df['home_team_defense'][i][0]
            df['home_R-CB'][i] = df['home_team_defense'][i][1]
            df['home_L-CB'][i] = df['home_team_defense'][i][2]
            df['home_LB'][i] = df['home_team_defense'][i][3]
            df['home_backline'][i] = 4
        elif(len(df['home_team_defense'][i])==3):
            df['home_RCB'][i] = df['home_team_defense'][i][0]
            df['home_CB'][i] = df['home_team_defense'][i][1]
            df['home_LCB'][i] = df['home_team_defense'][i][2]
            df['home_backline'][i] = 3
        else:
            df['home_RWB'][i] = df['home_team_defense'][i][0]
            df['home_RCB'][i] = df['home_team_defense'][i][1]
            df['home_CB'][i] = df['home_team_defense'][i][2]
            df['home_LCB'][i] = df['home_team_defense'][i][3]
            df['home_LWB'][i] = df['home_team_defense'][i][4]
            df['home_backline'][i] = 5

In [29]:
def add_away_posn(df):
    for i in range(0, len(df)):
        if(len(df['away_team_defense'][i])==4):
            df['away_RB'][i] = df['away_team_defense'][i][0]
            df['away_R-CB'][i] = df['away_team_defense'][i][1]
            df['away_L-CB'][i] = df['away_team_defense'][i][2]
            df['away_LB'][i] = df['away_team_defense'][i][3]
            df['away_backline'][i] = 4
        elif(len(df['away_team_defense'][i])==3):
            df['away_RCB'][i] = df['away_team_defense'][i][0]
            df['away_CB'][i] = df['away_team_defense'][i][1]
            df['away_LCB'][i] = df['away_team_defense'][i][2]
            df['away_backline'][i] = 3
        else:
            df['away_RWB'][i] = df['away_team_defense'][i][0]
            df['away_RCB'][i] = df['away_team_defense'][i][1]
            df['away_CB'][i] = df['away_team_defense'][i][2]
            df['away_LCB'][i] = df['away_team_defense'][i][3]
            df['away_LWB'][i] = df['away_team_defense'][i][4]
            df['away_backline'][i] = 5

In [20]:
defence = defence.reindex(columns=defence.columns.tolist() + [
    'home_RB',
    'home_R-CB',
    'home_L-CB',
    'home_LB',
    'home_RCB',
    'home_CB',
    'home_LCB',
    'home_RWB',
    'home_LWB',
    'away_RB',
    'away_R-CB',
    'away_L-CB',
    'away_LB',
    'away_RCB',
    'away_CB',
    'away_LCB',
    'away_RWB',
    'away_LWB',
    'home_backline',
    'away_backline'
])

In [30]:
add_home_posn(defence)
add_away_posn(defence)

In [31]:
defence

,match_id,home_team,away_team,match_date,home_team_defense,away_team_defense,home_RB,home_R-CB,home_L-CB,home_LB,home_RCB,home_CB,home_LCB,home_RWB,home_LWB,away_RB,away_R-CB,away_L-CB,away_LB,away_RCB,away_CB,away_LCB,away_RWB,away_LWB,home_backline,away_backline
0,22712,Burnley,Bournemouth,2018-05-13,"[MatthewLowton, KevinLong, JamesTarkowski, Ste...","[SteveCook, NathanAke, TyroneMings]",MatthewLowton,KevinLong,JamesTarkowski,StephenWard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SteveCook,NathanAke,TyroneMings,NaN,NaN,4,3
1,22713,Crystal Palace,West Brom,2018-05-13,"[AaronWanBissaka, JamesTomkins, MamadouSakho, ...","[AllanRomeoNyom, CraigDawson, AhmedHegazi, Kie...",AaronWanBissaka,JamesTomkins,MamadouSakho,PatrickvanAanholt,NaN,NaN,NaN,NaN,NaN,AllanRomeoNyom,CraigDawson,AhmedHegazi,KieranGibbs,NaN,NaN,NaN,NaN,NaN,4,4
2,22714,Huddersfield,Arsenal,2018-05-13,"[FlorentHadergjonaj, Zanka, ChristopherSchindl...","[HectorBellerin, ShkodranMustafi, RobHolding, ...",NaN,NaN,NaN,NaN,Zanka,ChristopherSchindler,TerenceKongolo,FlorentHadergjonaj,ChrisLwe,HectorBellerin,ShkodranMustafi,RobHolding,SeadKolasinac,NaN,NaN,NaN,NaN,NaN,5,4
3,22715,Liverpool,Brighton,2018-05-13,"[TrentAlexanderArnold, DejanLovren, VirgilvanD...","[EzequielSchelotto, ShaneDuffy, LewisDunk, Gae...",TrentAlexanderArnold,DejanLovren,VirgilvanDijk,AndrewRobertson,NaN,NaN,NaN,NaN,NaN,EzequielSchelotto,ShaneDuffy,LewisDunk,GaetanBong,NaN,NaN,NaN,NaN,NaN,4,4
4,22716,Man Utd,Watford,2018-05-13,"[MatteoDarmian, EricBailly, MarcosRojo, Ashley...","[DarylJanmaat, CraigCathcart, ChristianKabasel...",MatteoDarmian,EricBailly,MarcosRojo,AshleyYoung,NaN,NaN,NaN,NaN,NaN,DarylJanmaat,CraigCathcart,ChristianKabasele,JoseHolebas,NaN,NaN,NaN,NaN,NaN,4,4
5,22717,Newcastle,Chelsea,2018-05-13,"[DeAndreYedlin, JamaalLascelles, FlorianLejeun...","[CesarAzpilicueta, AndreasChristensen, GaryCah...",DeAndreYedlin,JamaalLascelles,FlorianLejeune,PaulDummett,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CesarAzpilicueta,AndreasChristensen,GaryCahill,NaN,NaN,4,3
6,22718,Southampton,Man City,2018-05-13,"[CdricSoares, JackStephens, MayaYoshida, Wesle...","[Danilo, JohnStones, AymericLaporte, FabianDelph]",NaN,NaN,NaN,NaN,JackStephens,MayaYoshida,WesleyHoedt,CdricSoares,RyanBertrand,Danilo,JohnStones,AymericLaporte,FabianDelph,NaN,NaN,NaN,NaN,NaN,5,4
7,22719,Swansea,Stoke,2018-05-13,"[AngelRangel, MikevanderHoorn, AlfieMawson, Ma...","[MoritzBauer, RyanShawcross, KurtZouma, ErikPi...",AngelRangel,MikevanderHoorn,AlfieMawson,MartinOlsson,NaN,NaN,NaN,NaN,NaN,MoritzBauer,RyanShawcross,KurtZouma,ErikPieters,NaN,NaN,NaN,NaN,NaN,4,4
8,22720,Spurs,Leicester,2018-05-13,"[KyleWalkerPeters, TobyAlderweireld, EricDier,...","[DannySimpson, WesMorgan, HarryMaguire, Christ...",KyleWalkerPeters,TobyAlderweireld,EricDier,DannyRose,NaN,NaN,NaN,NaN,NaN,DannySimpson,WesMorgan,HarryMaguire,ChristianFuchs,NaN,NaN,NaN,NaN,NaN,4,4
9,22721,West Ham,Everton,2018-05-13,"[DeclanRice, AngeloOgbonna, AaronCresswell]","[MichaelKeane, PhilJagielka, RamiroFunesMori]",NaN,NaN,NaN,NaN,DeclanRice,AngeloOgbonna,AaronCresswell,NaN,NaN,NaN,NaN,NaN,NaN,MichaelKeane,PhilJagielka,RamiroFunesMori,NaN,NaN,3,3


## Merge Match_IDs 

In [32]:
def label_splitter(df):
    df['label'] = df['label'].apply(lambda x: x.replace(",", ' '))
    df['label'] = df['label'].apply(lambda x: x.split('  '))
    df['score'] = df['label'].apply(lambda x: x[1])
    df['label'] = df['label'].apply(lambda x: x[0].split(' - '))
    df['home_team'] = df['label'].apply(lambda x: x[0])
    df['away_team'] = df['label'].apply(lambda x: x[1])
    return df

In [33]:
label_splitter(matches)

,status,roundId,gameweek,teamsData,seasonId,dateutc,winner,venue,wyId,label,date,referees,duration,competitionId,score,home_team,away_team
0,Played,4405654,38,"{'1646': {'scoreET': 0, 'coachId': 8880, 'side...",181150,2018-05-13 14:00:00,1659,Turf Moor,2500089,"[Burnley, AFC Bournemouth]","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 385705, 'role': 'referee'}, {'r...",Regular,364,1 - 2,Burnley,AFC Bournemouth
1,Played,4405654,38,"{'1628': {'scoreET': 0, 'coachId': 8357, 'side...",181150,2018-05-13 14:00:00,1628,Selhurst Park,2500090,"[Crystal Palace, West Bromwich Albion]","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 381851, 'role': 'referee'}, {'r...",Regular,364,2 - 0,Crystal Palace,West Bromwich Albion
2,Played,4405654,38,"{'1609': {'scoreET': 0, 'coachId': 7845, 'side...",181150,2018-05-13 14:00:00,1609,The John Smith's Stadium,2500091,"[Huddersfield Town, Arsenal]","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 384965, 'role': 'referee'}, {'r...",Regular,364,0 - 1,Huddersfield Town,Arsenal
3,Played,4405654,38,"{'1651': {'scoreET': 0, 'coachId': 8093, 'side...",181150,2018-05-13 14:00:00,1612,Anfield,2500092,"[Liverpool, Brighton & Hove Albion]","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 385704, 'role': 'referee'}, {'r...",Regular,364,4 - 0,Liverpool,Brighton & Hove Albion
4,Played,4405654,38,"{'1644': {'scoreET': 0, 'coachId': 93112, 'sid...",181150,2018-05-13 14:00:00,1611,Old Trafford,2500093,"[Manchester United, Watford]","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 381853, 'role': 'referee'}, {'r...",Regular,364,1 - 0,Manchester United,Watford
5,Played,4405654,38,"{'1613': {'scoreET': 0, 'coachId': 210700, 'si...",181150,2018-05-13 14:00:00,1613,St. James' Park,2500094,"[Newcastle United, Chelsea]","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 384888, 'role': 'referee'}, {'r...",Regular,364,3 - 0,Newcastle United,Chelsea
6,Played,4405654,38,"{'1625': {'scoreET': 0, 'coachId': 267136, 'si...",181150,2018-05-13 14:00:00,1625,St. Mary's Stadium,2500095,"[Southampton, Manchester City]","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 385911, 'role': 'referee'}, {'r...",Regular,364,0 - 1,Southampton,Manchester City
7,Played,4405654,38,"{'10531': {'scoreET': 0, 'coachId': 32573, 'si...",181150,2018-05-13 14:00:00,1639,Liberty Stadium,2500096,"[Swansea City, Stoke City]","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 378952, 'role': 'referee'}, {'r...",Regular,364,1 - 2,Swansea City,Stoke City
8,Played,4405654,38,"{'1631': {'scoreET': 0, 'coachId': 209010, 'si...",181150,2018-05-13 14:00:00,1624,Wembley Stadium,2500097,"[Tottenham Hotspur, Leicester City]","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 378951, 'role': 'referee'}, {'r...",Regular,364,5 - 4,Tottenham Hotspur,Leicester City
9,Played,4405654,38,"{'1623': {'scoreET': 0, 'coachId': 8541, 'side...",181150,2018-05-13 14:00:00,1633,London Stadium,2500098,"[West Ham United, Everton]","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 408156, 'role': 'referee'}, {'r...",Regular,364,3 - 1,West Ham United,Everton


In [34]:
matches = matches.replace({'home_team':{
        'Manchester United': 'Man Utd',
        'Tottenham Hotspur': 'Spurs',
        'West Ham United': 'West Ham',
        'Manchester City': 'Man City',
        'Brighton & Hove Albion': 'Brighton',
        'Stoke City': 'Stoke',
        'AFC Bournemouth': 'Bournemouth',
        'West Bromwich Albion': 'West Brom',
        'Leicester City': 'Leicester',
        'Swansea City': 'Swansea',
        'Huddersfield Town': 'Huddersfield',
        'Newcastle United': 'Newcastle'
    }, 'away_team': {
        'Manchester United': 'Man Utd',
        'Tottenham Hotspur': 'Spurs',
        'West Ham United': 'West Ham',
        'Manchester City': 'Man City',
        'Brighton & Hove Albion': 'Brighton',
        'Stoke City': 'Stoke',
        'AFC Bournemouth': 'Bournemouth',
        'West Bromwich Albion': 'West Brom',
        'Leicester City': 'Leicester',
        'Swansea City': 'Swansea',
        'Huddersfield Town': 'Huddersfield',
        'Newcastle United': 'Newcastle'
    }}
)

In [ ]:
matches.drop(columns=['label'], inplace=True)
matches['match'] = matches['home_team'] + '-' + matches['away_team']

In [ ]:
defence['match'] = defence['home_team'] + '-' + defence['away_team']

In [ ]:
match_def = pd.merge(right=matches, left=defence, right_on='match', left_on='match')

In [38]:
match_def.columns

Index(['match_id', 'home_team_x', 'away_team_x', 'match_date',
       'home_team_defense', 'away_team_defense', 'home_RB', 'home_R-CB',
       'home_L-CB', 'home_LB', 'home_RCB', 'home_CB', 'home_LCB', 'home_RWB',
       'home_LWB', 'away_RB', 'away_R-CB', 'away_L-CB', 'away_LB', 'away_RCB',
       'away_CB', 'away_LCB', 'away_RWB', 'away_LWB', 'home_backline',
       'away_backline', 'match', 'status', 'roundId', 'gameweek', 'teamsData',
       'seasonId', 'dateutc', 'winner', 'venue', 'wyId', 'date', 'referees',
       'duration', 'competitionId', 'score', 'home_team_y', 'away_team_y'],
      dtype='object')

In [ ]:
match_def.drop(columns=[
    'status', 'roundId', 'seasonId', 'date', 'match_date', 'winner',
    'match_id', 'duration', 'competitionId', 'home_team_y', 'away_team_y'
], inplace=True)

In [40]:
match_def.to_pickle('../data/matches_defence.pkl')

# Event Data Pre-Processing

## Normalizing Names 

In [ ]:
events_com['firstName'] = events_com['firstName'].swifter.set_npartitions(
    8).apply(lambda x: x.encode().decode('unicode_escape').replace('\xad',''))
events_com['lastName'] = events_com['lastName'].swifter.set_npartitions(
    8).apply(lambda x: x.encode().decode('unicode_escape').replace('\xad',''))

In [ ]:
players['firstName'] = players['firstName'].swifter.set_npartitions(
    8).apply(lambda x: x.encode().decode('unicode_escape').replace('\xad',''))
players['lastName'] = players['lastName'].swifter.set_npartitions(
    8).apply(lambda x: x.encode().decode('unicode_escape').replace('\xad',''))
players['playerName'] = players['firstName']+players['lastName']
players['playerName'] = players['playerName'].apply(lambda x: unidecode(x))

In [57]:
players['playerName'] = players['playerName'].apply(lambda x: x.replace('-', ''))

In [141]:
players['playerName'] = players['playerName'].apply(lambda x: x.replace(' ', ''))

In [142]:
players.to_pickle('../data/players.pkl')

## Modifying match_def 

In [92]:
match_def.columns

Index(['home_team_x', 'away_team_x', 'home_team_defense', 'away_team_defense',
       'home_RB', 'home_R-CB', 'home_L-CB', 'home_LB', 'home_RCB', 'home_CB',
       'home_LCB', 'home_RWB', 'home_LWB', 'away_RB', 'away_R-CB', 'away_L-CB',
       'away_LB', 'away_RCB', 'away_CB', 'away_LCB', 'away_RWB', 'away_LWB',
       'home_backline', 'away_backline', 'match', 'gameweek', 'teamsData',
       'dateutc', 'venue', 'wyId', 'referees', 'score', 'home_footedness'],
      dtype='object')

In [107]:
match_def_away = match_def[['wyId', 'away_team_x', 'away_team_defense', 'away_RB', 'away_R-CB', 'away_L-CB', 'away_LB', 'away_RCB',
       'away_CB', 'away_LCB', 'away_RWB', 'away_LWB', 'away_backline', 'match', 'gameweek', 'teamsData',
       'dateutc', 'venue', 'referees', 'score']]

In [108]:
match_def_away.columns = ['wyId', 'team', 'team_defense', 'RB', 'R-CB', 'L-CB', 'LB', 'RCB',
       'CB', 'LCB', 'RWB', 'LWB', 'backline', 'match', 'gameweek', 'teamsData',
       'dateutc', 'venue', 'referees', 'score']

In [110]:
match_def = match_def[['wyId', 'home_team_x', 'home_team_defense', 'home_RB', 'home_R-CB', 'home_L-CB', 'home_LB', 'home_RCB',
       'home_CB', 'home_LCB', 'home_RWB', 'home_LWB', 'home_backline', 'match', 'gameweek', 'teamsData',
       'dateutc', 'venue', 'referees', 'score']]

In [111]:
match_def.columns = ['wyId', 'team', 'team_defense', 'RB', 'R-CB', 'L-CB', 'LB', 'RCB',
       'CB', 'LCB', 'RWB', 'LWB', 'backline', 'match', 'gameweek', 'teamsData',
       'dateutc', 'venue', 'referees', 'score']

In [115]:
match_def = match_def.append(match_def_away, ignore_index=True)

In [119]:
match_def.sort_values(by=['gameweek'], inplace=True, ascending=False)

In [121]:
match_def.reset_index(drop=True)

,wyId,team,team_defense,RB,R-CB,L-CB,LB,RCB,CB,LCB,RWB,LWB,backline,match,gameweek,teamsData,dateutc,venue,referees,score
0,2500089,Burnley,"[MatthewLowton, KevinLong, JamesTarkowski, Ste...",MatthewLowton,KevinLong,JamesTarkowski,StephenWard,NaN,NaN,NaN,NaN,NaN,4,Burnley-Bournemouth,38,"{'1646': {'scoreET': 0, 'coachId': 8880, 'side...",2018-05-13 14:00:00,Turf Moor,"[{'refereeId': 385705, 'role': 'referee'}, {'r...",1 - 2
1,2500090,Crystal Palace,"[AaronWanBissaka, JamesTomkins, MamadouSakho, ...",AaronWanBissaka,JamesTomkins,MamadouSakho,PatrickvanAanholt,NaN,NaN,NaN,NaN,NaN,4,Crystal Palace-West Brom,38,"{'1628': {'scoreET': 0, 'coachId': 8357, 'side...",2018-05-13 14:00:00,Selhurst Park,"[{'refereeId': 381851, 'role': 'referee'}, {'r...",2 - 0
2,2500098,Everton,"[MichaelKeane, PhilJagielka, RamiroFunesMori]",NaN,NaN,NaN,NaN,MichaelKeane,PhilJagielka,RamiroFunesMori,NaN,NaN,3,West Ham-Everton,38,"{'1623': {'scoreET': 0, 'coachId': 8541, 'side...",2018-05-13 14:00:00,London Stadium,"[{'refereeId': 408156, 'role': 'referee'}, {'r...",3 - 1
3,2500097,Leicester,"[DannySimpson, WesMorgan, HarryMaguire, Christ...",DannySimpson,WesMorgan,HarryMaguire,ChristianFuchs,NaN,NaN,NaN,NaN,NaN,4,Spurs-Leicester,38,"{'1631': {'scoreET': 0, 'coachId': 209010, 'si...",2018-05-13 14:00:00,Wembley Stadium,"[{'refereeId': 378951, 'role': 'referee'}, {'r...",5 - 4
4,2500096,Stoke,"[MoritzBauer, RyanShawcross, KurtZouma, ErikPi...",MoritzBauer,RyanShawcross,KurtZouma,ErikPieters,NaN,NaN,NaN,NaN,NaN,4,Swansea-Stoke,38,"{'10531': {'scoreET': 0, 'coachId': 32573, 'si...",2018-05-13 14:00:00,Liberty Stadium,"[{'refereeId': 378952, 'role': 'referee'}, {'r...",1 - 2
5,2500095,Man City,"[Danilo, JohnStones, AymericLaporte, FabianDelph]",Danilo,JohnStones,AymericLaporte,FabianDelph,NaN,NaN,NaN,NaN,NaN,4,Southampton-Man City,38,"{'1625': {'scoreET': 0, 'coachId': 267136, 'si...",2018-05-13 14:00:00,St. Mary's Stadium,"[{'refereeId': 385911, 'role': 'referee'}, {'r...",0 - 1
6,2500094,Chelsea,"[CesarAzpilicueta, AndreasChristensen, GaryCah...",NaN,NaN,NaN,NaN,CesarAzpilicueta,AndreasChristensen,GaryCahill,NaN,NaN,3,Newcastle-Chelsea,38,"{'1613': {'scoreET': 0, 'coachId': 210700, 'si...",2018-05-13 14:00:00,St. James' Park,"[{'refereeId': 384888, 'role': 'referee'}, {'r...",3 - 0
7,2500093,Watford,"[DarylJanmaat, CraigCathcart, ChristianKabasel...",DarylJanmaat,CraigCathcart,ChristianKabasele,JoseHolebas,NaN,NaN,NaN,NaN,NaN,4,Man Utd-Watford,38,"{'1644': {'scoreET': 0, 'coachId': 93112, 'sid...",2018-05-13 14:00:00,Old Trafford,"[{'refereeId': 381853, 'role': 'referee'}, {'r...",1 - 0
8,2500092,Brighton,"[EzequielSchelotto, ShaneDuffy, LewisDunk, Gae...",EzequielSchelotto,ShaneDuffy,LewisDunk,GaetanBong,NaN,NaN,NaN,NaN,NaN,4,Liverpool-Brighton,38,"{'1651': {'scoreET': 0, 'coachId': 8093, 'side...",2018-05-13 14:00:00,Anfield,"[{'refereeId': 385704, 'role': 'referee'}, {'r...",4 - 0
9,2500091,Arsenal,"[HectorBellerin, ShkodranMustafi, RobHolding, ...",HectorBellerin,ShkodranMustafi,RobHolding,SeadKolasinac,NaN,NaN,NaN,NaN,NaN,4,Huddersfield-Arsenal,38,"{'1609': {'scoreET': 0, 'coachId': 7845, 'side...",2018-05-13 14:00:00,The John Smith's Stadium,"[{'refereeId': 384965, 'role': 'referee'}, {'r...",0 - 1


In [122]:
match_def.to_pickle('../data/match+def_lineup.pkl')

## Adding Footedness 

In [3]:
match_def = pd.read_pickle('../data/matches/match+def_lineup.pkl')

In [4]:
a = match_def[['team_defense', 'backline']].explode('team_defense')

In [3]:
players = pd.read_pickle('../data/players.pkl')

In [ ]:
p = players[['playerName', 'foot']]

In [ ]:
b = pd.merge(right=p, left=a, right_on='playerName', left_on='team_defense', how='left')

In [162]:
b[b['foot'].isna()]['team_defense'].unique()

array(['RamiroFunesMori', 'KurtZouma', 'Danilo', 'CesarAzpilicueta',
       'EzequielSchelotto', 'GaetanBong', 'HectorBellerin', 'AhmedHegazi',
       'JamaalLascelles', 'AngelRangel', 'Zanka', 'ChrisLwe',
       'EricBailly', 'MarcosRojo', 'CdricSoares', 'AngeloOgbonna',
       'HctorBellern', 'DavinsonSanchez', 'JavierManquillo', 'TommySmith',
       'Bruno', 'GatanBong', 'NicolsOtamendi', 'CsarAzpilicueta',
       'AntonioRdiger', 'JosHolebas', 'SamusColeman', 'AllanRomoNyom',
       'NathanAk', 'JosephGomez', 'AlbertoMoreno', 'LuisAntonioValencia',
       'VictorLindelf', 'DavinsonSnchez', 'NicolasOtamendi',
       'NachoMonreal', 'FedericoFernndez', 'SebastianPrdl',
       'CedricSoares', 'JoelMatip', 'MiguelBritos', 'VictorLindelof',
       'JamesCollins', 'CucoMartina', 'DavidLuiz', 'MollaWagu',
       'JrmyPied', 'ChancelMbemba', 'PabloZabaleta', 'KikoFemenia',
       'CheikhouKouyat', 'KikoFemena', 'JoseFonte', 'JosFonte',
       'JesusGamez'], dtype=object)

In [65]:
b.loc[b['team_defense'].isin([
    'CesarAzpilicueta',
    'CsarAzpilicueta',
    'HectorBellerin',
    'HctorBellern',
    'EricBailly',
    'KurtZouma',
    'Danilo',
    'EzequielSchelotto',
    'AhmedHegazi',
    'JamaalLascelles',
    'AngelRangel',
    'Zanka',
    'CdricSoares',
    'CedricSoares',
    'DavinsonSanchez', 
    'JavierManquillo',
    'TommySmith',
    'NicolsOtamendi',
    'NicolasOtamendi',
    'AntonioRdiger',
    'AntonioRudiger',
    'SamusColeman',
    'AllanRomoNyom',
    'JosephGomez',
    'LuisAntonioValencia',
    'VictorLindelf',
    'VictorLindelof',
    'DavinsonSnchez',
    'FedericoFernndez',
    'SebastianPrdl',
    'JoelMatip',
    'JamesCollins',
    'DavidLuiz',
    'CucoMartina',
    'MollaWagu',
    'JoseFonte',
    'JosFonte',
    'JrmyPied',
    'ChancelMbemba',
    'PabloZabaleta',
    'KikoFemenia',
    'CheikhouKouyat',
    'KikoFemena',
    'Bruno',
    'JesusGamez'
]), 'foot'] ='right'

In [66]:
b.loc[b['team_defense'].isin([
    'MarcosRojo',
    'RamiroFunesMori',
    'GaetanBong',
    'GatanBong',
    'ChrisLwe',
    'AngeloOgbonna', 
    'JosHolebas',
    'NathanAk',
    'AlbertoMoreno',
    'NachoMonreal',
    'MiguelBritos'    
]), 'foot'] = 'left'

In [67]:
b['foot'].isna().sum()

0

In [68]:
b.index = a.index

In [ ]:
f = [list(b['foot'][i]) for i in a.index.unique()]
f = pd.Series(f)
match_def['footedness'] = f.values
match_def['footedness'] = match_def['footedness'].apply(lambda x: '-'.join(x))

In [195]:
match_def.reset_index(drop=True, inplace=True)
match_def.rename(columns={'wyId': 'matchId'})

In [72]:
match_def.to_pickle('../data/matches/match+def_lineup+footedness_ver2.pkl')

In [ ]:
events_com['playerName'] = events_com['firstName']+events_com['lastName']
events_com.drop(columns=[
    'currentTeamId', 'passportArea', 'weight', 'firstName', 'lastName',
    'middleName', 'birthDate', 'height', 'role', 'birthArea', 'shortName',
    'currentNationalTeamId'
], inplace=True)
events_com.rename(columns={'id': 'eventId', 'wyId': 'playerId'})

In [201]:
events_com.to_pickle('../data/events+player.pkl')

## Mapping Event Tags

In [53]:
event = pd.read_pickle('../data/events+player.pkl')

In [ ]:
tag_key = pd.read_csv('../data/events/tags2name.csv')

In [ ]:
k = list(zip(tag_key['Tag'], tag_key['Description']))

In [50]:
def unpack(x):
    x = [list(x[i].values()) for i in range(0, len(x))]
    return x

In [61]:
def functag2name(x):
    tag_names = list()
    for val in x:
        val = int(val)
        val_pair = [item for item in k if item[0]==val]
        tag_names.append(val_pair[0][1])
    return tag_names

In [ ]:
event['tags'] = event['tags'].swifter.set_npartitions(8).apply(lambda x: unpack(x))
event['tags'] = event['tags'].swifter.set_npartitions(8).apply(lambda x: list(chain(*x)))

In [ ]:
event['tags'] = event['tags'].swifter.set_npartitions(8).apply(lambda x: functag2name(x))

In [63]:
event.to_pickle('../data/events+players.pkl')

## Converting Positions Co-ordinates 

In [49]:
events = pd.read_pickle('../data/events+players.pkl')

In [33]:
def clean_coordinates(x):
    x = [list(d.values()) for d in x]
    x = [l[::-1] for l in x]
    return x

In [34]:
events['positions'] = events['positions'].swifter.set_npartitions(8).apply(lambda x: clean_coordinates(x))

In [ ]:
events['positions'] = events['positions'].swifter.set_npartitions(8).apply(lambda x: [np.multiply([-1.04, 0.68], i) for i in x])
events['positions'] = events['positions'].swifter.set_npartitions(8).apply(lambda x: [np.subtract([0, 68], i) for i in x])

In [62]:
events['playerName'] = events["playerName"].apply(lambda x: unidecode(x))
events['playerName'] = events["playerName"].apply(lambda x: x.replace(' ',''))
events['playerName'] = events["playerName"].apply(lambda x: x.replace('-',''))

In [63]:
def roundcoords(x):
    final_roundedcoords = list()
    for l in x:
        roundedcoords = [np.round(val,decimals=2) for val in l]
        final_roundedcoords.append(roundedcoords)
    return final_roundedcoords

In [64]:
events['positions'] = events['positions'].apply(lambda x: roundcoords(x))

In [65]:
df_players = pd.read_pickle('../data/players.pkl')

In [66]:
# Adding Roles
roles_temp = df_players['role'].values

roles = list()
for i in roles_temp:
    roles.append(i['code3'])
    

players_roles = list(zip(roles,df_players['wyId'],df_players['playerName']))

df_players_roles = pd.DataFrame(players_roles,columns = ['role','playerId','playerName1'])

df_events_roles = events.merge(df_players_roles, left_on = 'playerId', right_on = 'playerId'  )

df_events_roles.drop(['playerName1'], axis = 1, inplace = True)

df_events_roles.to_pickle('../data/events/events_v2.pkl')

# Result

1. Use **events_plot-a-match.pkl** to plot match events from same match simultaneously<br><br>
2. Use **events_v2.pkl** to plot match events from different matches cumulatively over a season - used in all the analysis<br><br>
3. Use **match+def_lineup+footedness_ver2.pkl** for Defensive lineups of teams in each match from the PL 17-18 season<br><br>
**Note:** All event coordinate data has origin on the bottom left corner of the pitch when viewed horizontally and x = [0,104], y= [0,68]